## IGNORE THIS NOTEBOOK
I am still working on getting a Qiskit tutorial set up so this isn't quite ready for a walk-thru yet.

In [1]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity

## Quantum Circuits
Quantum circuits are made using registers, which are created either when initiated or by using the add_register command.

In [2]:
q0 = QuantumRegister(2, 'q0')
q1 = QuantumRegister(2, 'q1')
c0 = ClassicalRegister(2, 'c0')
c1 = ClassicalRegister(2, 'c1')

In [3]:
circ = QuantumCircuit(q0, q1)
#circ.x(q0[0])
circ.x(q0[1])
circ.x(q1[0])
circ.draw()

In [4]:
circ2 = QuantumCircuit()
circ2.add_register(q0)
circ2.add_register(q1)
circ2.x(q0[1])
circ2.x(q1[0])
circ2.draw()

In [5]:
# We are copying circ to circ3 and then adding an additional register, q3. 
# We now have an object that is (3) registers x (2) quibits
from copy import deepcopy

q3 = QuantumRegister(2, 'q3')
circ3 = deepcopy(circ)
circ3.add_register(q3)
circ3.draw()

Note: The circuit drawer has the last register added at the bottom. If we add a new register it will add it to the bottom of the circuit.

### Extending a circuit
In many situatios you may have two circuits taht you want to concatenate to form a new circuit. This is very useful when one circuit has no measurements, and the final circuit represents a measurement.

In [6]:
meas = QuantumCircuit(q0, q1, c0, c1)
meas.measure(q0, c0)
meas.measure(q1, c1)

qc = circ + meas

qc.draw()

In [7]:
meas2 = QuantumCircuit()
meas2.add_register(q0)
meas2.add_register(q1)
meas2.add_register(c0)
meas2.add_register(c1)
meas2.measure(q0, c0)
meas2.measure(q1, c1)

qc2 = circ2 + meas2

qc2.draw()

In [8]:
circ4 = QuantumCircuit(q1)
circ4.x(q1)
circ4.draw()

In [9]:
circ5 = QuantumCircuit(q3)
circ5.h(q3)
circ5.draw()

In [10]:
(circ4+circ5).draw()

We have also overloaded += to the QuantumCircuit object:

In [11]:
circ4 += circ5
circ4.draw()

## Outcomes of Quantum Circuits
In the circuit output, the most significant bit (MSB) is to the left, and the least significant bit (LSB) is to the right (i.e., we follow little-endian ordering from computer science). In this example:

In [12]:
circ.draw()

qubit register $Q_{0}$ is prepared in the state $|10\rangle$ and $Q_{1}$ is in the state $|01\rangle$, giving a total state $|0110\rangle\langle Q1 \otimes Q0\rangle$. __Note__: The tensor order in Qiskit goes as $Q_{n}\otimes\dots Q_{1}\otimes Q_{0}$. That is the four-qubit statevector of length 16, with sixth element `(int('0110',2))=6)` being one. Note the element count starts from zero.

In [13]:
# Output at this point is all zero. We have given the list of gates to 
# apply to the circuit but have not ACTUALL applied the gates. 
backend_sim = BasicAer.get_backend('statevector_simulator')
result = execute(circ, backend_sim).result()
state = result.get_statevector(circ)
print(state)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


To check the fidelity of this state with the `basis_state` in Qiskit Terrra, use:

In [14]:
state_fidelity(basis_state('0110', 4), state)

1.0

We can also use Qiskit Terra to make the unitary operator representing the circuit (provided there are no measurements). This will be a $16 \otimes 16$ matrix equal to $I\otimes X \otimes X \otimes I$. To check this is correct, we can use the `Pauli` class and the `process_fidelity` function.

In [15]:
backend_sim = BasicAer.get_backend('unitary_simulator')
result = execute(circ, backend_sim).result()
unitary = result.get_unitary(circ)
process_fidelity(Pauli(label='IXXI').to_matrix(), unitary)

(1+0j)

To map the information of the quantum state to the classical world, we use the example with measurement `qc`:

In [16]:
qc.draw()

This will map the quantum state to the classical world. Since the state has no superpositions, it will be deterministic and equal to `'01 10'`. Here a space is used to separate the registers.

In [17]:
backend_sim = BasicAer.get_backend('qasm_simulator')
result = execute(qc, backend_sim).result()
counts = result.get_counts(qc)
print(counts)

{'01 10': 1024}


To show that it does not matter how you add the registers, we run the same as above on the second example circuit:

In [18]:
backend_sim = BasicAer.get_backend('statevector_simulator')
result = execute(circ2, backend_sim).result()
states = result.get_statevector(circ2)

backend_sim = BasicAer.get_backend('qasm_simulator')
result = execute(qc2, backend_sim).result()
counts = result.get_counts(qc2)

backend_sim = BasicAer.get_backend('unitary_simulator')
result = execute(circ2, backend_sim).result()
unitary = result.get_unitary(circ2)

In [19]:
print(counts)

{'01 10': 1024}


In [20]:
state_fidelity(basis_state('0110', 4), state)

1.0

In [21]:
process_fidelity(Pauli(label='IXXI').to_matrix(), unitary)

(1+0j)

## Counting circuit resources
A `QuantumCircuit` object provides methods for inquiring its resource use. This includes the number of qubits, operations, and a few other things.

In [22]:
q = QuantumRegister(6)
circuit = QuantumCircuit(q)
circuit.h(q[0])
circuit.ccx(q[0], q[1], q[2])
circuit.cx(q[1], q[3])
circuit.x(q)
circuit.h(q[2])
circuit.h(q[3])
circuit.draw()

In [23]:
# total number of operations in the circuit. No unrolling is done.
circuit.size()

11

In [24]:
# depth of circuit (number of ops on the critical path)
circuit.depth()

5

In [25]:
# number of qubits in the circuit
circuit.width()

6

In [26]:
# a breakdown of operations by type
circuit.count_ops()

{'h': 3, 'ccx': 1, 'cx': 1, 'x': 6}

In [27]:
# number of unentangled subcircuits in this circuit.
# each subcircuit can in principle be executed on a different quantum processor!
circuit.num_tensor_factors()

3